In [9]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import pymetis
import networkx as nx
import time
from networkx.algorithms import community
from random import shuffle
import math
import torch
import torch.nn as nn
import torch_geometric as tg
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops, degree
from torch.nn import init
import pdb
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
from torch_geometric.data import Data
import torch.optim as optim
import pywt
from scipy.stats import norm
import scipy.interpolate as interp
import collections

In [7]:
def open_data(file_path):
    file = open(file_path,"rb")
    raw_data = pickle.load(file)
    
    return raw_data

In [8]:
def read_data(file_path):
    with h5py.File(file_path, 'r') as f:
    # 查看文件中所有的数据集名称
        dataset_names = list(f.keys())
        print("Datasets in the file:", dataset_names)
        data_dict = {}
        for name in dataset_names:
            if f[name].shape == (): 
                data = f[name][()] 
            else:
                data = f[name][:]
        
            data_dict[name] = data
            
        return data_dict

In [4]:
def save_data(data, file_path):
    with open(file_path , 'wb') as f:
        pickle.dump(data,f)
        f.close()

In [5]:
def save_data_ash5(data, file_path):
    with h5py.File(file_path, 'w') as f:
        keys =list(data.keys())
        for i in range(len(keys)):
            f.create_dataset(keys[i], data=data[keys[i]])

In [20]:
def renumber_subgraph(nodes, edge_index):
    unique_nodes = torch.unique(nodes)
    new_node_ids = torch.arange(len(unique_nodes))
    node_mapping = {old_id.item(): new_id.item() for old_id, new_id in zip(unique_nodes, new_node_ids)}
    
    new_edge_index = torch.tensor([
        [node_mapping[edge_index[0, i].item()], node_mapping[edge_index[1, i].item()]]
        for i in range(edge_index.size(1))
    ]).t()
    
    return new_edge_index

In [7]:
def precompute_dist_data(edge_pairs):
#距离是1/实际距离
    graph = nx.Graph()
    edge_pairs = edge_pairs.transpose(1,0).tolist()
    graph.add_edges_from(edge_pairs)
    H = nx.Graph()
    
    for node in sorted(graph.nodes):
        H.add_node(node)
        
    for edge in graph.edges:
        H.add_edge(*edge)  
        
    n = np.unique(edge_pairs).shape[0]
    dists_array = np.zeros((n, n))
        # dists_dict = nx.all_pairs_shortest_path_length(graph,cutoff=approximate if approximate>0 else None)
        # dists_dict = {c[0]: c[1] for c in dists_dict}
    dists_dict =dict(nx.all_pairs_shortest_path_length(H))
    
    for i, node_i in enumerate(H.nodes()):
        shortest_dist = dists_dict[node_i]
        for j, node_j in enumerate(H.nodes()):
            dist = shortest_dist.get(node_j, -1)
            if dist!=-1:
                dists_array[i, j] = 1 / (dist + 1)
    return torch.tensor(dists_array)

In [8]:
def get_random_anchorset(n,c=0.5): #n是一个图的所有点数量
    m = int(np.log2(n))
    copy = int(c*m)
    anchorset_id = []
    for i in range(m):
        anchor_size = int(n/np.exp2(i + 1))
        for j in range(copy):
            anchorset_id.append(np.random.choice(n, size=anchor_size, replace=False))
    return anchorset_id

In [9]:
def get_dist_max(edge_pairs, max_ach_num, max_subgraph_node_num):
    dist = precompute_dist_data(edge_pairs)
    n=np.unique(edge_pairs).shape[0]
    anchorset_id = get_random_anchorset(n,c=0.5)
    dist_max = torch.zeros((max_subgraph_node_num, max_ach_num))
    dist_argmax = torch.zeros((max_subgraph_node_num, max_ach_num)).long()
    for i in range(len(anchorset_id)):
        temp_id = torch.as_tensor(anchorset_id[i], dtype=torch.long)
        dist_temp = dist[:, temp_id]
        dist_max_temp, dist_temp_argmax = torch.max(dist_temp, dim=-1)#找到每一行距离最大的anchor对应的列索引
        dist_argmax_temp = temp_id[dist_temp_argmax]
        #-------------------------------------------------------
        dist_max[:dist.shape[0],i] = dist_max_temp
        dist_argmax[:dist.shape[0],i] = dist_argmax_temp
        
    return dist_max, dist_argmax

In [10]:
def Dist_Max_Dist_Argmax_Prepare(edge_index, max_ach_num, max_subgraph_node_num, layer_num):
    dists_max_sets = []
    dists_argmax_sets = []
    for i in range(layer_num):
        dists_max, dists_argmax = get_dist_max(edge_index, max_ach_num, max_subgraph_node_num)
        dists_max_sets.append(dists_max)
        dists_argmax_sets.append(dists_argmax)
    return torch.stack(dists_max_sets), torch.stack(dists_argmax_sets)

In [11]:
def subgraph_information(city_names, root_path):
    edge_pair_dictionary = {}
    max_ach_num = 0
    max_subgraph_node_num = 0 
    
    for name in city_names:
        edge_pairs = []
        dirs = os.listdir(root_path + name + "/edge_pair/")
        
        #-------------获取每个字图的edge pairs----------
        edge_pairs = [torch.tensor(open_data(root_path + name + "/edge_pair/" + each_file),dtype=torch.long) for each_file in dirs]
        edge_pair_dictionary[name + "_edge_pair"] = edge_pairs
        
        #------------获取每个子图的节点-----------------
        comm_node_list = [torch.unique(edge_pair, sorted = True) for edge_pair in edge_pairs]
        edge_pair_dictionary[name + "_nodes"] = comm_node_list
        
        #--------------计算子图的节点数-----------------
        graph_num = [sub_graph.shape[0] for sub_graph in comm_node_list]
        edge_pair_dictionary[name + "subgraph_node_num"] = graph_num
        
        #---------Calculate the anchor set num of each comm----------------
        ach_set_nums = [int(0.5*int(np.log2(node_num.shape[0])))* int(np.log2(node_num.shape[0]))
                        for node_num in comm_node_list]
        edge_pair_dictionary[name + "_anchor_set_num"] = ach_set_nums
        
        #---------获取每个城市的节点数-----------------
        city_node_num = sum([comm_node.shape[0] for comm_node in comm_node_list])
        edge_pair_dictionary[name + "_city_node_num"] = city_node_num
        
        if max_ach_num <= max(ach_set_nums):
            max_ach_num = max(ach_set_nums)
        if max_subgraph_node_num <= max(graph_num):
            max_subgraph_node_num = max(graph_num)
        
    return edge_pair_dictionary, max_ach_num, max_subgraph_node_num

In [12]:
city_names = ["Barcelona", "Antwerp"]
root_path = "D:/ThesisData/processed data/"

In [13]:
edge_pair_dictionary, max_ach_num, max_subgraph_node_num = subgraph_information(city_names, root_path)

In [11]:
#all_nodes = edge_pair_dictionary["Barcelona_nodes"]+edge_pair_dictionary["Antwerp_nodes"]
#min_id = [torch.min(comm_nodes).item() for comm_nodes in all_nodes]
#print(min_id)

[42002, 33203, 12432, 22822, 43181, 12062, 8418, 16634, 26714, 6879, 36, 35521, 13, 8239, 69882, 56331, 60477, 82300, 67556, 76823, 60767, 41276, 79371, 57, 105, 33793, 27355, 53652, 53002, 49452, 22, 4, 27314, 48475, 15186, 24000, 55558, 189, 162, 84056, 63355, 80753]


In [16]:
max_ach_num =72

In [6]:
city_name = "Antwerp"
data_dict = read_data(root_path + city_name + "/input_target/input_target.h5")

Datasets in the file: ['city_node_num', 'period_template', 'target_template', 'trend_template']


In [9]:
data_dict['trend_template'].shape

(71, 20, 111260, 6)

In [14]:
#trend_patterns Shape:100//seq_len, seq_len, graph_num, feature_num
city_node_num = data_dict['city_node_num']
trend_patterns = torch.tensor(data_dict["trend_template"], dtype = torch.float)
periodic_patterns = torch.tensor(data_dict["period_template"], dtype = torch.float)  #[:, :, :city_node_num, :]

def comm_data_aggregation(patterns, subgraph_nodes):
    subgraph_parts = torch.zeros((patterns.shape[0], patterns.shape[1], patterns.shape[2], 1))
    for i in range(patterns.shape[0]):
        row_indicies = [torch.isin(patterns[i, :, :, 0], subgraph_nodes[j]) for j in range(len(subgraph_nodes))]
        for j in range(len(row_indicies)):
            
            #repeated_subgraph_nodes = subgraph_nodes[j].repeat(torch.tensor(subgraph_parts[i, :, :, 0][row_indicies[j]].shape)
                                                               #// torch.tensor(subgraph_nodes[j].shape))
            subgraph_parts[i, :, :, 0][row_indicies[j]] = j #repeated_subgraph_nodes.to(dtype=torch.float)
        
    patterns = torch.cat((patterns, subgraph_parts), dim = 3)      
    
    return patterns


    

In [15]:
subgraph_edge_pairs = edge_pair_dictionary[city_name + "_edge_pair"]
subgraph_nodes = edge_pair_dictionary[city_name + "_nodes"]
new_data_dict ={}
new_data_dict['target_template']=torch.tensor(data_dict["target_template"], dtype = torch.float)

pattern_list = [trend_patterns, periodic_patterns]
for i in range(len(pattern_list)):
    comm_based_patterns = comm_data_aggregation(pattern_list[i], subgraph_nodes)
    if i == 0:
        new_data_dict["trend_template"] =comm_based_patterns
    else:
        new_data_dict["period_template"] =comm_based_patterns

In [22]:
new_data_dict["target_template"]

tensor([[[[4.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [5.0000e+00, 6.8232e-03, 0.0000e+00, 0.0000e+00, 1.5861e-02,
           0.0000e+00],
          [6.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          ...,
          [1.1142e+05, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [1.1142e+05, 1.9495e-03, 0.0000e+00, 0.0000e+00, 5.6647e-02,
           0.0000e+00],
          [1.1142e+05, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00]],

         [[4.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [5.0000e+00, 6.8232e-03, 0.0000e+00, 0.0000e+00, 1.5861e-02,
           0.0000e+00],
          [6.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          ...,
          [1.1142e+05, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00],
          [1.1142e

In [17]:
save_path = root_path + city_name + "/input_target/comm_based_input_target.h5"
save_data_ash5(new_data_dict, save_path)

In [14]:
#-----------Calculate distance_matrix----------
def dist_max_argmax_cal(subgraph_nodes, subgraph_edge_pairs, max_ach_num, max_subgraph_node_num, layer_num):
    subgraph_dist_max = []
    subgraph_dist_argmax = []
    for i in range(len(subgraph_nodes)):
        new_edge_index = renumber_subgraph(subgraph_nodes[i], subgraph_edge_pairs[i])
        dist_max_tensors, dist_argmax_tensors = Dist_Max_Dist_Argmax_Prepare(new_edge_index, max_ach_num, max_subgraph_node_num, layer_num)
        subgraph_dist_max.append(dist_max_tensors)
        subgraph_dist_argmax.append(dist_argmax_tensors)
    
    #torch.tensor(subgraph_dist_max) Shape: SubgraphNum, LayerNum, Max_Subgraph_Node_num, Max_Ach_Num
    return torch.stack(subgraph_dist_max).to(dtype=torch.float), torch.stack(subgraph_dist_argmax).to(dtype=torch.float)

In [19]:
layer_num = 3
city_name = "Barcelona"
subgraph_edge_pairs = edge_pair_dictionary[city_name + "_edge_pair"]
subgraph_nodes = edge_pair_dictionary[city_name + "_nodes"]

subgraph_dist_max, subgraph_dist_argmax =dist_max_argmax_cal(subgraph_nodes, subgraph_edge_pairs,
                                                             max_ach_num, max_subgraph_node_num, layer_num)
dist_dictionary = {"dist_max": subgraph_dist_max, "dist_argmax": subgraph_dist_argmax}

save_path = root_path + city_name + "/input_target/dist_dictionary.h5"
save_data(dist_dictionary, save_path)


KeyboardInterrupt: 